# Generación de textos con Deep Learning

Las redes neuronales recurrentes también pueden usarse como modelos generativos. Esto significa que, además de ser utilizados para modelos predictivos (hacer predicciones), pueden aprender las secuencias de un problema y luego generar secuencias plausibles completamente nuevas para el dominio del problema.  En este proyecto, vamos a descubrir cómo crear un modelo generativo de texto, carácter por carácter, utilizando las redes neuronales recurrentes de LSTM en Python con Keras.


# **Descripción del problema: Generación de texto**

## **¿Qué utilidad tienen los modelos generativos?**
Estos modelos permiten en base a un conjunto de datos aprender y generar datos siguiendo secuencias aprendidas bajo los datos. Esto se utiliza mucho en **phising**. En base a mensajes preestablecidos, se generan emails, cuentas de usuarios, mensajes en aplicaciones de mensajería... etc.. para engañar al usuarios simulando que se encuentra en una conversión real. O también  en los **chatbots** para replicar la comunicación humana sin necesidad de tener a una persona al otro lado y facilitar la interacción por ejemplo con una empresa las 24 horas del día.

## **Enunciado**
Para implementar un modelo generativo sencillo vamos a seleccionar un libro de texto sencillo de nuestra infancia para utilizarlo como base de aprendizaje y a partir de ahí con Redes LSTM generar un modelo que pueda generar un texto a partir de su aprendizaje. Aprenderemos las dependencias entre los caracteres y las probabilidades condicionales de los personajes en las secuencias para que a su vez podamos generar secuencias de caracteres totalmente nuevas y originales.

Como datos de la práctica se entregan los siguientes ficheros:
- Cuento de Los 3 cerditos que sirva como base de aprendizaje (puedes utilizar el libro o texto que consideres)
- El proyecto realizado y explicado como ejemplo con una pequeña red neuronal


# Red Neuronal Recurrente LSTM más grande
Ahora vamos ha hacer los mismo creando una red mucho más grande. Mantendremos el mismo número de unidades de memoria en 256, pero añadiremos una segunda capa.

In [ ]:
#Define la LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

También cambiaremos el nombre del peso de los puntos de control para que podamos determinar la diferencia entre los pesos de esta red y el anterior (agregando la palabra más grande en el nombre del archivo):

**filename="pesos-los3grande-303-30-2.3081.hdf5"**

Por último, aumentaremos el número de epoch de formación de 50 a 300 y reduciremos el tamaño del lote de 128 a 64 para dar a la red más oportunidades de actualización y aprendizaje. El código completo sería (con todos los pasos vistos):

In [ ]:
#Grande LSTM Network para generar texto 3 cerditos
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
#Cargamos el texto y lo pasamos a minuscula
filename = "los3.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
#Crear mapeo de caracteres únicos a enteros, y un mapeo inverso
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
#Sumarizamos los datos cargados
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total caracters: ", n_chars)
print("Total vocabulario: ", n_vocab)
#Preparar el conjunto de datos de entrada para los pares de salida codificados como enteros
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total patrones: ", n_patterns)
#Remodelar X para que sea [muestras, pasos de tiempo, características]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
#Normalizacion
X = X / float(n_vocab)
#Codificacion en caliente con la variable de salida
y = np_utils.to_categorical(dataY)
#Se define el LSTM modelo
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
#Se define el checkpoint
filepath="pesos-los3grandes-303-{epoch:02d}-{loss:.4f}.hdf5";
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
#Ajuste del modelo
model.fit(X, y, epochs=300, batch_size=64, callbacks=callbacks_list)

La ejecución de este ejemplo lleva algún tiempo, dependiendo de tu ordenador. Después de ejecutar este ejemplo, se puede lograr una pérdida de aproximadamente 0.018. Por ejemplo, el mejor resultado que obtuve al ejecutar este modelo se almacenó en un archivo de punto de control con el nombre:

In [ ]:
filename="pesos-los3grandes-303-150-0.0105.hdf5"

Como en el punto anterior, podemos utilizar este mejor modelo de la ejecución para generar texto. El único cambio que necesitamos hacer en el script de generación de texto del punto anterior está en la especificación de la topología de la red y desde qué archivo sembrar los pesos de la red. El código completo sería (con todos los pasos vistos):

In [ ]:
#Carga de la red LSTM grande para generar texto
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
#Cargamos el texto y lo pasamos a minuscula
filename = "los3.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
#Crear mapeo de caracteres únicos a enteros, y un mapeo inverso
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
#Sumarizamos los datos cargados
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
#Preparar el conjunto de datos de entrada para los pares de salida codificados como enteros
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    n_patterns = len(dataX)

print("Total Patterns: ", n_patterns)
#Remodelar X para que sea [muestras, pasos de tiempo, características]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
#Normalizacion
X = X / float(n_vocab)
#Codificacion en caliente de la variable de salida
y = np_utils.to_categorical(dataY)
#Define la LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
#Carga de los pesos de la red
filename="pesos-los3grandes-303-150-0.0105.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
#Toma una semilla aleatoria
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Semilla:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
#Genera los carácteres
for i in range(1000):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print("\nHecho.")

Un ejemplo de ejecución de este script de generación de texto produce la salida a continuación. El texto semilla elegido al azar fue:

Semilla: *" l más pequeño-, la paja es blanda y se puede sujetar con facilidad. terminaré muy pronto y podré ir "*

El texto generado con la semilla (limpiado para presentación) fue:

*a jugar. el hermano mediano decidió que su casa sería de madera: - puedo encontrar un montón de madera por los alrededores - explicó a sus hermanos, - construiré mi casa en un santiamén con todos estos troncos y me iré también a jugar. cuando las tres casitas estuvieron terminadas, los cerditos cantaban y bailaban en la puerta, felices por haber acabado con el problema: -¡quién teme al lobo feroz, al lobo, al lobo! - ¡quién teme al lobo feroz, al lobo feroz! - cantaban desde dentro los cerditos. el lobo estaba realmente enfadado y hambriento, y ahora deseaba comerse a los tres cerditos más que nunca, y frente a la puerta dijo: - ¡cerditos, abridme la puerta! - no, no, no, no te vamos a abrir. - pues si no me abrís... ¡soplaré y soplaré y la casita derribaré! y se puso a soplar tan fuerte como el viento de invierno. sopló y sopló, pero la casita de ladrillos era muy resistente y no conseguía derribarla. decidió trepar por la pared y entrar por la chimenea. se deslizó hacia abajo
Hecho.*

Podemos ver que generalmente hay menos errores de ortografía y el texto parece más realista. Estos son mejores resultados, pero hay todavía se podría mejorar mucho más.